In [13]:
import pandas as pd

# 엑셀 파일 불러오기
df = pd.read_csv("../전처리/data_d.csv")  # 또는 pd.read_csv("파일명.csv")

# 모든 열이 NaN인 행 제거
df_cleaned = df.dropna(how='all')

# 인덱스 초기화 (기존 인덱스 무시하고 0부터 재정렬)
df_cleaned.reset_index(drop=True, inplace=True)

# 저장 (선택)
df_cleaned.to_csv("../전처리/data_d.csv", index=False)  # 또는 to_csv(...)

In [14]:
import chardet

with open("food_음식점데이터.csv", "rb") as f:
    result = chardet.detect(f.read())
    print(result)

{'encoding': 'EUC-KR', 'confidence': 0.99, 'language': 'Korean'}


In [15]:
restaurants = pd.read_csv("food_음식점데이터.csv",encoding='EUC-KR')
print(restaurants.columns.tolist())
restaurants.dropna(subset=['위도', '경도'], inplace=True)

['ID', '명칭', '우편번호', '관리자', '문의 및 안내', '주소', '위도', '경도', '개요', '주차 시설', '어린이놀이방', '영업시간', '쉬는날', '대표메뉴', '취급메뉴', '금연/흡연', '신용카드정보', '포장가능', '예약안내', '상세정보', '카페/음식점']


In [16]:
import pandas as pd
from geopy.distance import geodesic

# ------------------------
# 1. 데이터 불러오기
# ------------------------
attractions = pd.read_csv("data_d.csv")  # 관광지
# restaurants = pd.read_csv("food_음식점데이터.csv")
cafes = pd.read_csv("food_카페데이터.csv")              # 카페
# 한글 컬럼명을 영문으로 변경
attractions.rename(columns={'위도': 'lat', '경도': 'lng','명칭':'name'}, inplace=True)
restaurants.rename(columns={'위도': 'lat', '경도': 'lng','명칭':'name'}, inplace=True)
cafes.rename(columns={'위도': 'lat', '경도': 'lng','명칭':'name'}, inplace=True)
# ------------------------
# 2. 거리 기반 필터 함수
# ------------------------
def filter_by_distance(df, center_lat, center_lng, radius_km):
    filtered = []
    for _, row in df.iterrows():
        dist = geodesic((center_lat, center_lng), (row['lat'], row['lng'])).km
        if dist <= radius_km:
            row_dict = row.to_dict()
            row_dict['distance_km'] = round(dist, 2)
            filtered.append(row_dict)
    return pd.DataFrame(filtered)

# ------------------------
# 3. 추천 로직
# ------------------------
def recommend_chain(start_lat, start_lng):
    print("🚩 출발지 기준 관광지 추천 중...")
    nearby_attractions = filter_by_distance(attractions, start_lat, start_lng, radius_km=10)
    
    if nearby_attractions.empty:
        print("❌ 100km 이내 관광지가 없습니다.")
        return
    
    top_attraction = nearby_attractions.iloc[0]
    print(nearby_attractions)
    # print(f"✅ 추천 관광지: {top_attraction['name']} ({top_attraction['distance_km']} km)")
    
    print("\n🍽 관광지 기준 음식점 추천 중...")
    nearby_restaurants = filter_by_distance(restaurants, top_attraction['lat'], top_attraction['lng'], radius_km=10)
    
    if nearby_restaurants.empty:
        print("❌ 20분 이내 음식점이 없습니다.")
        return
    
    top_restaurant = nearby_restaurants.iloc[0]
    print(f"✅ 추천 음식점: {top_restaurant['name']} ({top_restaurant['distance_km']} km)")

    print("\n☕ 음식점 기준 카페 추천 중...")
    nearby_cafes = filter_by_distance(cafes, top_restaurant['lat'], top_restaurant['lng'], radius_km=5)
    
    if nearby_cafes.empty:
        print("❌ 10분 이내 카페가 없습니다.")
        return
    
    top_cafe = nearby_cafes.iloc[0]
    print(f"✅ 추천 카페: {top_cafe['name']} ({top_cafe['distance_km']} km)")

# ------------------------
# 사용 예시
# ------------------------
# 예: 서울 시청 출발
recommend_chain(start_lat=33.511111, start_lng=	126.492778)

🚩 출발지 기준 관광지 추천 중...
                     id       name category        theme  \
0   KCLANPO23N000000368    제주관광안내소      관광지    관광안내소/매표소   
1   KCLANPO23N000000562     도두항유람선      관광지        일반관광지   
2   KCLANPO23N000000563        해신사      관광지        천연기념물   
3   KCLANPO23N000000654     제주공룡랜드      관광지  테마공원/대형놀이공원   
4   KCLANPO23N000000710     삼양해수욕장      관광지         해수욕장   
..                  ...        ...      ...          ...   
88  KCLANPO23N000043167    노형수퍼마켓2      관광지        일반관광지   
89  KCLANPO23N000043260  제주전농로벚꽃거리      관광지        유명관광지   
90  KCLANPO23N000044514      캐니언파크      관광지          동물원   
91  KCLANPO23N000045412    숲속야영장휴림      관광지          야영장   
92  KCLANPO23N000046372    이호테우말등대      관광지        일반관광지   

                     address address_detail        lat         lng  tags/0  \
0      제주특별자치도 제주시 용담이동  483        용두암길 15  33.514842  126.511421     매표소   
1   제주특별자치도 제주시 도두일동  268455       도두항서길 28  33.506616  126.465123   일반관광지   
2     제주특별자치도 제주시 화북일동  

In [ ]:
# 거리 계산 후 전부 정렬해서 상위 5개 확인
def print_nearest(df, center_lat, center_lng):
    distances = []
    for _, row in df.iterrows():
        if pd.isna(row['lat']) or pd.isna(row['lng']):
            continue
        try:
            dist = geodesic((center_lat, center_lng), (row['lat'], row['lng'])).km
            distances.append((row['name'], dist))
        except:
            continue

    sorted_result = sorted(distances, key=lambda x: x[1])
    for name, d in sorted_result[:5]:
        print(f"{name}: {round(d, 2)} km")

# 확인 실행
print_nearest(attractions, 37.511111, 126.9780)  # 서울 시청

최영장군사당: 404.32 km
상추자전망대: 404.53 km
제주올레23: 404.62 km
후포해변: 404.69 km
예초리기정길끝: 404.79 km


In [12]:
for radius in [50, 100, 500, 1000, 2000]:
    print(f"🔎 반경 {radius}km 내 검색 중...")
    result = filter_by_distance(attractions, 37.5665, 126.9780, radius_km=radius)
    if not result.empty:
        print(f"✅ {radius}km 이내에 {len(result)}개의 관광지가 있습니다.")
        break
else:
    print("❌ 어느 반경에서도 관광지를 찾을 수 없습니다.")

🔎 반경 50km 내 검색 중...


NameError: name 'filter_by_distance' is not defined

In [18]:
print("음식점 NaN 수:\n", restaurants[['lat', 'lng']].isna().sum())
# print("카페 NaN 수:\n", cafes[['위도', '경도']].isna().sum())

음식점 NaN 수:
 lat    0
lng    0
dtype: int64


In [22]:
import pandas as pd
from geopy.distance import geodesic

# ------------------------
# 1. 데이터 불러오기
# ------------------------
attractions = pd.read_csv("data_d.csv")
restaurants = pd.read_csv("food_음식점데이터.csv",encoding='EUC-KR')
cafes = pd.read_csv("food_카페데이터.csv", encoding='utf-8-sig')

# 한글 컬럼명을 영문으로 변경
attractions.rename(columns={'위도': 'lat', '경도': 'lng', '명칭': 'name'}, inplace=True)
restaurants.rename(columns={'위도': 'lat', '경도': 'lng', '명칭': 'name'}, inplace=True)
cafes.rename(columns={'위도': 'lat', '경도': 'lng', '명칭': 'name'}, inplace=True)

# NaN 제거
attractions.dropna(subset=['lat', 'lng'], inplace=True)
restaurants.dropna(subset=['lat', 'lng'], inplace=True)
cafes.dropna(subset=['lat', 'lng'], inplace=True)

# ------------------------
# 2. 브루트포스 거리 계산 함수
# ------------------------
def find_closest_place(df, center_lat, center_lng):
    closest = None
    min_distance = float('inf')

    for _, row in df.iterrows():
        try:
            dist = geodesic((center_lat, center_lng), (row['lat'], row['lng'])).km
            if dist < min_distance:
                min_distance = dist
                closest = row.to_dict()
                closest['distance_km'] = round(dist, 2)
        except:
            continue

    return closest

# ------------------------
# 3. 추천 체인 로직
# ------------------------
def recommend_chain(start_lat, start_lng):
    print("🚩 출발지 기준 관광지 추천 중...")
    top_attraction = find_closest_place(attractions, start_lat, start_lng)

    if not top_attraction:
        print("❌ 관광지를 찾을 수 없습니다.")
        return

    print(f"✅ 추천 관광지: {top_attraction['name']} ({top_attraction['distance_km']} km)")

    print("\n🍽 관광지 기준 음식점 추천 중...")
    top_restaurant = find_closest_place(restaurants, top_attraction['lat'], top_attraction['lng'])

    if not top_restaurant:
        print("❌ 가까운 음식점을 찾을 수 없습니다.")
        return

    print(f"✅ 추천 음식점: {top_restaurant['name']} ({top_restaurant['distance_km']} km)")

    print("\n☕ 음식점 기준 카페 추천 중...")
    top_cafe = find_closest_place(cafes, top_restaurant['lat'], top_restaurant['lng'])

    if not top_cafe:
        print("❌ 가까운 카페를 찾을 수 없습니다.")
        return

    print(f"✅ 추천 카페: {top_cafe['name']} ({top_cafe['distance_km']} km)")

# ------------------------
# 사용 예시
# ------------------------
recommend_chain(start_lat=33.511111, start_lng=	126.492778)
# print_nearest(attractions, 37.511111, 126.9780)  # 서울 시청

🚩 출발지 기준 관광지 추천 중...
✅ 추천 관광지: 제주카페거리 (0.9 km)

🍽 관광지 기준 음식점 추천 중...
✅ 추천 음식점: 해녀잠수촌 (0.09 km)

☕ 음식점 기준 카페 추천 중...
✅ 추천 카페: 홉히 (0.28 km)


In [29]:
from geopy.distance import geodesic

def score_places(df, user_lat, user_lng, user_category, max_distance_km=10, w_content=0.7, w_distance=0.3):
    scored = []

    for _, row in df.iterrows():
        try:
            lat, lng = row['lat'], row['lng']
            if pd.isna(lat) or pd.isna(lng):
                continue

            distance = geodesic((user_lat, user_lng), (lat, lng)).km
            if distance > max_distance_km:
                continue  # 너무 먼 장소는 제외

            # 콘텐츠 점수 (카테고리 일치 여부)
            content_score = 1 if row.get('theme_category', '') == user_category else 0

            # 거리 점수 계산 (1 - 거리/최대거리)
            distance_score = 1 - (distance / max_distance_km)

            # 최종 점수
            final_score = w_content * content_score + w_distance * distance_score

            row_data = row.to_dict()
            row_data['distance_km'] = round(distance, 2)
            row_data['content_score'] = content_score
            row_data['distance_score'] = round(distance_score, 2)
            row_data['final_score'] = round(final_score, 4)
            scored.append(row_data)

        except:
            continue

    return pd.DataFrame(scored).sort_values('final_score', ascending=False).head(5)

In [38]:
# 관광지에서 추천 상위 5개 뽑기
recommended = score_places(
    df=attractions,
    user_lat=33.511111,
    user_lng=126.492778,
    user_category='',  # 예: 사용자가 '자연'을 선호한다고 응답
    max_distance_km=10,
    w_content=0.7,
    w_distance=0.3
)

print(recommended[['name', 'theme_category', 'distance_km', 'content_score', 'distance_score', 'final_score']])

       name theme_category  distance_km  content_score  distance_score  \
75   제주카페거리             기타         0.90              0            0.91   
21  용두암해안도로           관광명소         1.06              0            0.89   
36     선사무덤        역사/문화유산         1.28              0            0.87   
12    선사주거지        역사/문화유산         1.38              0            0.86   
0   제주관광안내소      관광안내소/매표소         1.78              0            0.82   

    final_score  
75       0.2730  
21       0.2681  
36       0.2615  
12       0.2585  
0        0.2466  


In [35]:
from geopy.distance import geodesic

def score_based_recommendation(df, user_lat, user_lng, selected_category, max_distance_km=10,
                                w_content=0.7, w_distance=0.3, top_n=5):
    results = []

    # 1. 거리 최대값 계산 (브루트포스 중 최장거리 찾기용)
    max_possible_distance = 0
    for _, row in df.iterrows():
        if pd.isna(row['lat']) or pd.isna(row['lng']):
            continue
        try:
            dist = geodesic((user_lat, user_lng), (row['lat'], row['lng'])).km
            if dist > max_possible_distance:
                max_possible_distance = dist
        except:
            continue

    # 2. 점수 계산
    for _, row in df.iterrows():
        if pd.isna(row['lat']) or pd.isna(row['lng']):
            continue

        try:
            distance_km = geodesic((user_lat, user_lng), (row['lat'], row['lng'])).km
        except:
            continue

        if distance_km > max_distance_km:
            continue  # 너무 멀면 제외

        # 거리 점수
        normalized_distance = distance_km / max_possible_distance if max_possible_distance else 1
        distance_score = 1 - normalized_distance

        # 콘텐츠 점수: 사용자가 선택한 카테고리와 일치할 때만 점수 부여
        content_score = 1.0 if row.get('theme') == selected_category else 0.0

        # 최종 점수
        total_score = w_content * content_score + w_distance * distance_score

        result = row.to_dict()
        result['distance_km'] = round(distance_km, 2)
        result['score'] = round(total_score, 4)
        results.append(result)

    # 3. 상위 N개 정렬 반환
    results = sorted(results, key=lambda x: x['score'], reverse=True)
    return results[:top_n]

In [36]:
recommendations = score_based_recommendation(
    df=attractions,
    user_lat=33.511111,
    user_lng=126.492778,
    selected_category="해수욕장",  # 사용자가 선택한 소주제
    max_distance_km=10,     # 10km 이내만 필터링
    w_content=0.7,
    w_distance=0.3,
    top_n=5
)

for rec in recommendations:
    print(f"✅ {rec['name']} | 거리: {rec['distance_km']}km | 점수: {rec['score']}")

✅ 이호테우해수욕장 | 거리: 3.96km | 점수: 0.9778
✅ 알작지해변 | 거리: 5.23km | 점수: 0.9708
✅ 삼양해수욕장 | 거리: 8.79km | 점수: 0.9508
✅ 제주카페거리 | 거리: 0.9km | 점수: 0.295
✅ 용두암해안도로 | 거리: 1.06km | 점수: 0.294


In [ ]:
import pandas as pd

# 관광지 데이터 불러오기

df = pd.read_csv("data2.csv",encoding='EUC-KR')
# "theme" 열에 있는 '폭포' → '강' 으로 통일
df['theme'] = df['theme'].str.replace("폭포", "강", regex=False)
# 분류 함수 정의
def classify_theme_category(theme):
    if pd.isna(theme):
        return "기타"
    if any(kw in theme for kw in ["산", "바다", "공원", "드라이브", "동굴", "산책", "강"]):
        return "자연/경치형"
    elif any(kw in theme for kw in ["사찰", "유적지", "고택", "미술관", "촬영지", "기념관"]):
        return "역사/문화형"
    elif any(kw in theme for kw in ["테마파크", "체험마을", "스포츠시설"]):
        return "액티비티형"
    else:
        return "기타"

# 분류 적용
df["theme_category"] = df["theme"].apply(classify_theme_category)

# 결과 저장 (선택)
df.to_csv("관광지_분류완료2.csv", index=False)

In [2]:
import pandas as pd

# 두 개의 CSV 파일 불러오기
df1 = pd.read_csv("final.csv")
df2 = pd.read_excel("성시경 211to422필터링.xlsx")

# 두 데이터를 세로로 이어붙이기 (행 기준 합치기)
merged_df = pd.concat([df1, df2], ignore_index=True)

# 결과 저장
merged_df.to_csv("merged_result.csv", index=False)

In [57]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 관광지 데이터 불러오기
df = pd.read_csv("merged_result.csv")  # 파일명에 맞게 수정
names = df['name'].fillna("").astype(str)

# TF-IDF 벡터화 (문자 단위 n-gram 사용하면 더 잘 작동)
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 3))
tfidf_matrix = vectorizer.fit_transform(names)

# Cosine similarity 계산
cosine_sim = cosine_similarity(tfidf_matrix)

# 유사한 항목끼리 그룹으로 묶기 (임계값 0.6 이상이면 같은 그룹으로 판단)
threshold = 0.4
visited = set()
groups = []

for i in range(len(names)):
    if i in visited:
        continue
    group = [names[i]]
    visited.add(i)
    for j in range(i + 1, len(names)):
        if cosine_sim[i, j] >= threshold:
            group.append(names[j])
            visited.add(j)
    if len(group) > 1:
        groups.append(group)

# 결과 출력
for idx, group in enumerate(groups, 1):
    print(f"\n📦 그룹 {idx}:")
    for name in group:
        print(" -", name)


📦 그룹 1:
 - 동산물감귤체험농장
 - 봉봉감귤체험농장
 - 제주왕감귤체험농장
 - 걸세악감귤체험농장

📦 그룹 2:
 - 도깨비공원
 - 도깨비도로

📦 그룹 3:
 - 갯깍주상절리대
 - 대포주상절리대

📦 그룹 4:
 - 귀덕해안도로
 - 함덕해안도로

📦 그룹 5:
 - 김녕미로공원
 - 녹차미로공원

📦 그룹 6:
 - 산굼부리
 - 굼부리

📦 그룹 7:
 - 수월봉
 - 수월봉지질트레일

📦 그룹 8:
 - 종달리해안도로
 - 종달리해변

📦 그룹 9:
 - 제주민속마을
 - 제주민속촌

📦 그룹 10:
 - 비자림
 - 제주키위비자림농원

📦 그룹 11:
 - 제주동백마을
 - 제주동백수목원

📦 그룹 12:
 - 지질트레일
 - 수월봉지질트레일


In [ ]:
import pandas as pd

# 1. 분류된 기준 데이터 로드
classified_df = pd.read_excel("211to422.xlsx")  # theme, theme_category 포함된 CSV
# 열 이름: ['id', 'name', 'theme', 'theme_category']

# 2. 원본 전체 데이터 로드 (채워야 하는 대상)
original_df = pd.read_csv("data_d.csv")  # theme, theme_category가 누락되었거나 비어 있는 데이터

# 3. ID 기준 병합하여 theme, theme_category 채우기
merged_df = original_df.merge(classified_df[['id', 'theme', 'theme_category']], on='id', how='left', suffixes=('', '_filled'))

# 4. 누락된 값이 있는 경우 채우기
merged_df['theme'] = merged_df['theme'].combine_first(merged_df['theme_filled'])
merged_df['theme_category'] = merged_df['theme_category'].combine_first(merged_df['theme_category_filled'])

# 5. 불필요한 보조 열 제거
merged_df.drop(columns=['theme_filled', 'theme_category_filled'], inplace=True)

# 6. 결과 저장 (선택사항)
merged_df.to_csv("merged_data_with_theme.csv", index=False)

print("✅ theme 및 theme_category 열이 ID 기준으로 병합 및 채워졌습니다.")